# Samplers and Efficiency

**Module 6.4, Lesson 2** | CourseAI

In the lesson, you learned that the model predicts noise and the sampler decides what to do with that prediction. DDPM takes 1000 tiny steps. DDIM predicts x₀ and leaps. DPM-Solver reads the road ahead. All use the same trained model—no retraining required.

**What you will do:**
- Swap samplers on the same pre-trained model and compare quality and speed across DDPM, DDIM, and DPM-Solver
- Verify that DDIM is deterministic (same seed = same image) while DDPM is stochastic
- Explore the quality-vs-steps curve for DPM-Solver and find the sweet spot
- Inspect DDIM intermediate latents at each step and observe the coarse-to-fine trajectory

**For each exercise, PREDICT the output before running the cell.**

This is a STRETCH notebook. The exercises build understanding of sampler mechanisms using `diffusers` schedulers—not implementing samplers from scratch. The focus is on comparing behavior, inspecting intermediate states, and verifying the "same model, different sampler" insight.

**Estimated time:** 30–45 minutes.

---

## Setup

Run this cell to install dependencies and import everything.

In [ ]:
!pip install -q diffusers transformers accelerate

import torch
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image
import time

# Reproducible results
torch.manual_seed(42)
np.random.seed(42)

# Nice plots
plt.style.use('dark_background')
plt.rcParams['figure.figsize'] = [10, 4]

# Device setup
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
dtype = torch.float16 if device.type == 'cuda' else torch.float32
print(f'Using device: {device}')
if device.type == 'cuda':
    print(f'GPU: {torch.cuda.get_device_name(0)}')
    print(f'VRAM: {torch.cuda.get_device_properties(0).total_mem / 1024**3:.1f} GB')

print('\nSetup complete.')

## Shared Helpers

Load the Stable Diffusion pipeline once and define helper functions for generating images and displaying results. All exercises share the same model weights.

In [ ]:
from diffusers import StableDiffusionPipeline, DDPMScheduler, DDIMScheduler, DPMSolverMultistepScheduler

model_id = 'stable-diffusion-v1-5/stable-diffusion-v1-5'

# Load the pipeline once. We will swap the scheduler for each exercise.
print('Loading Stable Diffusion pipeline...')
pipe = StableDiffusionPipeline.from_pretrained(
    model_id,
    torch_dtype=dtype,
    safety_checker=None,
    requires_safety_checker=False,
)
pipe = pipe.to(device)
print(f'Pipeline loaded on {device}.')

# Save the scheduler config so we can create fresh schedulers from it.
scheduler_config = pipe.scheduler.config


def latent_to_pil(pipe, latent):
    """Decode a latent tensor to a PIL image using the pipeline's VAE."""
    with torch.no_grad():
        image = pipe.vae.decode(latent / pipe.vae.config.scaling_factor).sample
    image = image.detach().cpu().float().squeeze(0).permute(1, 2, 0).numpy()
    image = np.clip((image + 1.0) / 2.0, 0.0, 1.0)
    return Image.fromarray((image * 255).astype(np.uint8))


def show_images(images, titles, figsize=None):
    """Display a list of PIL images side by side."""
    n = len(images)
    if figsize is None:
        figsize = (5 * n, 5)
    fig, axes = plt.subplots(1, n, figsize=figsize)
    if n == 1:
        axes = [axes]
    for ax, img, title in zip(axes, images, titles):
        ax.imshow(np.array(img))
        ax.set_title(title, fontsize=10)
        ax.axis('off')
    plt.tight_layout()
    plt.show()


print('Helpers defined.')

---

## Exercise 1: Same Model, Different Sampler [Guided]

The lesson's central claim: the same trained model produces comparable results at 1000, 50, and 20 steps with different samplers. No retraining. Same weights. Different walkers.

In this exercise, you will generate an image with:
1. **DDPMScheduler** at 1000 steps (the original algorithm from Module 6.2)
2. **DDIMScheduler** at 50 steps (predict-x₀-then-jump, 20× fewer steps)
3. **DPMSolverMultistepScheduler** at 20 steps (higher-order solver, 50× fewer steps)

All three use the **exact same U-Net weights**, the **same prompt**, and the **same starting noise** (same seed). The only variable is the scheduler.

**Before running, predict:**
- Will the 20-step DPM-Solver image look significantly worse than the 1000-step DDPM image?
- How much faster will DPM-Solver at 20 steps be compared to DDPM at 1000 steps?
- Will the three images look identical, or will there be visible differences?

In [ ]:
prompt = "a cat sitting on a beach at sunset"
guidance_scale = 7.5
seed = 42

# Define the three configurations: (scheduler class, num_steps, label)
configs = [
    (DDPMScheduler, 1000, 'DDPM (1000 steps)'),
    (DDIMScheduler, 50, 'DDIM (50 steps)'),
    (DPMSolverMultistepScheduler, 20, 'DPM-Solver (20 steps)'),
]

images = []
timings = []

for scheduler_cls, num_steps, label in configs:
    print(f'\n--- {label} ---')

    # Swap the scheduler. One line. No retraining.
    pipe.scheduler = scheduler_cls.from_config(scheduler_config)
    print(f'Scheduler: {pipe.scheduler.__class__.__name__}')

    # Generate with the same seed for fair comparison.
    generator = torch.Generator(device=device).manual_seed(seed)
    start = time.time()

    result = pipe(
        prompt,
        num_inference_steps=num_steps,
        guidance_scale=guidance_scale,
        generator=generator,
    )

    elapsed = time.time() - start
    timings.append(elapsed)
    images.append(result.images[0])

    print(f'Time: {elapsed:.1f}s')
    print(f'U-Net forward passes: {num_steps * 2} (steps × 2 for CFG)')

# Display all three side by side
titles = [
    f'DDPM (1000 steps)\n{timings[0]:.1f}s',
    f'DDIM (50 steps)\n{timings[1]:.1f}s',
    f'DPM-Solver (20 steps)\n{timings[2]:.1f}s',
]
show_images(images, titles, figsize=(15, 5))

# Timing comparison
print(f'\n=== Timing Summary ===')
print(f'DDPM (1000 steps):      {timings[0]:>6.1f}s  (baseline)')
print(f'DDIM (50 steps):        {timings[1]:>6.1f}s  ({timings[0]/timings[1]:.0f}× speedup)')
print(f'DPM-Solver (20 steps):  {timings[2]:>6.1f}s  ({timings[0]/timings[2]:.0f}× speedup)')
print()
print('Same model. Same weights. Same prompt. Same starting noise.')
print('The ONLY difference: how the noise prediction was used to take steps.')

### What Just Happened

You generated three images with the **exact same model weights**, the same prompt, and the same starting noise. The only variable was the scheduler:

- **DDPM at 1000 steps:** The original algorithm from Module 6.2. Takes a tiny step at each of the 1000 timesteps, adding fresh noise at every step. Slow but faithful to the original formulation.
- **DDIM at 50 steps:** Predicts x₀ from the noise prediction, then leaps to a distant timestep using the closed-form formula. 20× fewer steps, comparable quality. Deterministic (no noise injected per step).
- **DPM-Solver at 20 steps:** A higher-order ODE solver that evaluates the model at multiple points to estimate trajectory curvature, enabling even larger steps. 50× fewer steps than DDPM.

The images are not identical—different samplers follow different paths through the noise-to-data space. But the **quality** is comparable, despite the massive difference in step counts and compute time.

The key insight: swapping `pipe.scheduler` is one line of code. No retraining. No weight changes. The model’s job (predict noise) never changes. The sampler’s job (decide how to step) is the only variable.

---

## Exercise 2: DDIM Determinism vs DDPM Stochasticity [Guided]

The lesson explained that DDIM with σ=0 is fully deterministic: same starting noise = same image, every time. DDPM is stochastic: it injects fresh random noise at every step, so different random states produce different images.

In this exercise, you will:
1. Generate 3 images with **DDIM** using the same seed before each run. Verify they are pixel-identical.
2. Generate 3 images with **DDPM** using a single seed (set once, not re-seeded between runs). The generator's internal state advances after each run, producing different images.

**Before running, predict:**
- Will DDIM with the same seed produce the exact same image every time? (Think about the σ=0 determinism from the lesson.)
- Will DDPM produce different images when the generator state has advanced between runs? (Think about the fresh noise z injected at every step—the σ_t · z term in the DDPM reverse formula.)

In [ ]:
prompt = "a lighthouse on a rocky cliff"
seed = 123
num_runs = 3

# ---- Part 1: DDIM (deterministic) ----
print('=== DDIM (50 steps, σ=0) ===')
pipe.scheduler = DDIMScheduler.from_config(scheduler_config)

ddim_images = []
for i in range(num_runs):
    generator = torch.Generator(device=device).manual_seed(seed)
    result = pipe(
        prompt,
        num_inference_steps=50,
        guidance_scale=7.5,
        generator=generator,
    )
    ddim_images.append(result.images[0])
    print(f'  Run {i+1} complete')

# Check pixel-level identity
ddim_arrays = [np.array(img) for img in ddim_images]
ddim_1_vs_2 = np.array_equal(ddim_arrays[0], ddim_arrays[1])
ddim_1_vs_3 = np.array_equal(ddim_arrays[0], ddim_arrays[2])
print(f'\nDDIM: Run 1 == Run 2? {ddim_1_vs_2}')
print(f'DDIM: Run 1 == Run 3? {ddim_1_vs_3}')
if ddim_1_vs_2 and ddim_1_vs_3:
    print('All 3 DDIM images are PIXEL-IDENTICAL. Same seed = same image, guaranteed.')

show_images(
    ddim_images,
    [f'DDIM Run {i+1}' for i in range(num_runs)],
    figsize=(15, 5),
)

In [ ]:
# ---- Part 2: DDPM (stochastic) ----
print('=== DDPM (50 steps, stochastic) ===')
# Note: DDPM was designed for 1000 steps, but we use 50 here for speed.
# Quality will be lower, but the stochasticity point still holds.
# We also use fewer steps so the exercise runs in a reasonable time.
# The quality difference you see is about step count (DDPM outside its
# designed range), NOT about stochastic vs deterministic per se.
pipe.scheduler = DDPMScheduler.from_config(scheduler_config)

ddpm_images = []

# Seed the generator ONCE. Each pipeline call consumes random state
# (for z_T AND for per-step noise injection). After the first run,
# the generator's internal state has advanced, so the second and third
# runs draw DIFFERENT random numbers—producing different images.
generator = torch.Generator(device=device).manual_seed(seed)

for i in range(num_runs):
    # Do NOT re-seed between runs! The whole point is that DDPM's
    # stochastic noise injection means different generator states
    # produce different images.
    result = pipe(
        prompt,
        num_inference_steps=50,
        guidance_scale=7.5,
        generator=generator,
    )
    ddpm_images.append(result.images[0])
    print(f'  Run {i+1} complete')

# Check pixel-level identity
ddpm_arrays = [np.array(img) for img in ddpm_images]
ddpm_1_vs_2 = np.array_equal(ddpm_arrays[0], ddpm_arrays[1])
ddpm_1_vs_3 = np.array_equal(ddpm_arrays[0], ddpm_arrays[2])
print(f'\nDDPM: Run 1 == Run 2? {ddpm_1_vs_2}')
print(f'DDPM: Run 1 == Run 3? {ddpm_1_vs_3}')

# Compute difference magnitude
diff_1_2 = np.abs(ddpm_arrays[0].astype(float) - ddpm_arrays[1].astype(float))
print(f'Mean pixel difference (Run 1 vs Run 2): {diff_1_2.mean():.2f} / 255')
print(f'Max pixel difference (Run 1 vs Run 2): {diff_1_2.max():.0f} / 255')

show_images(
    ddpm_images,
    [f'DDPM Run {i+1}' for i in range(num_runs)],
    figsize=(15, 5),
)

### What Just Happened

You verified two fundamental properties of the samplers:

- **DDIM is deterministic** (σ=0): all three runs with the same seed produced pixel-identical images. The predict-x₀-then-jump mechanism uses no random noise during sampling. Given the same starting noise z_T, the trajectory to z_0 is fully determined by the model's predictions at each step. This is why DDIM is valuable for reproducibility, A/B testing, and interpolation in z_T space.

- **DDPM is stochastic**: the three runs produced different images even though they started from the same generator seed. Why? Because the generator was seeded once and its internal state advanced after each run. DDPM injects fresh random noise at every step—the σ_t · z term in the reverse formula. Each pipeline call consumed dozens of random draws (one per step for the noise injection, plus one for z_T). By the second run, the generator was in a different state, producing a different z_T and different per-step noise. The result: three distinct images from the same starting seed.

- **Note on quality:** DDPM was designed for 1000 steps. At 50 steps (used here for speed), quality is lower. That quality difference is about step count, not about stochastic vs deterministic. The stochasticity claim is about *diversity*—different images from different random states—which holds regardless of step count.

The temperature analogy from **Sampling and Generation** applies directly: DDIM is temperature=0 (deterministic, always picks the "most likely" path). DDPM is temperature>0 (stochastic, explores diverse paths). Same model, same destination, but DDPM wanders while DDIM strides.

---

## Exercise 3: Step Count Exploration [Supported]

The lesson claimed that advanced samplers have a **sweet spot**: too few steps degrades quality, but adding steps beyond the sweet spot yields diminishing returns. DPM-Solver at 200 steps is not meaningfully better than at 25.

Your task: generate images at **5, 10, 20, 50, 100, and 200 steps** with DPM-Solver, display them in a grid, and time each generation. Then answer:
- At what step count does quality become acceptable?
- At what step count does increasing steps stop improving quality noticeably?
- Plot generation time vs step count—is the relationship linear?

**Hints:**
- Use `DPMSolverMultistepScheduler` for all runs
- Use the same seed for fair comparison
- Store both images and timings in lists
- Use `matplotlib` to create a time-vs-steps plot

In [ ]:
prompt = "a cozy cabin in a snowy forest at night, warm light from windows"
seed = 77
guidance_scale = 7.5

step_counts = [5, 10, 20, 50, 100, 200]
step_images = []
step_timings = []

# TODO: Loop over step_counts. For each step count:
#   1. Set the scheduler to DPMSolverMultistepScheduler
#   2. Create a generator with the same seed
#   3. Time the generation with pipe()
#   4. Append the image and timing to the lists
#
# Look at Exercise 1 for the pattern of setting a scheduler and calling pipe().

for n in step_counts:
    pipe.scheduler = DPMSolverMultistepScheduler.from_config(scheduler_config)
    generator = torch.Generator(device=device).manual_seed(seed)

    start = time.time()
    # YOUR CODE HERE: call pipe() to generate the image
    result = None
    elapsed = time.time() - start

    if result is not None:
        step_images.append(result.images[0])
        step_timings.append(elapsed)
        print(f'{n:>3d} steps: {elapsed:.1f}s')
    else:
        print(f'{n:>3d} steps: TODO - fill in the pipe() call above')
        break

In [ ]:
# Display the image grid (only runs if images were generated)
if len(step_images) == len(step_counts):
    fig, axes = plt.subplots(2, 3, figsize=(15, 10))
    for ax, img, n, t in zip(axes.flat, step_images, step_counts, step_timings):
        ax.imshow(np.array(img))
        ax.set_title(f'{n} steps ({t:.1f}s)', fontsize=11)
        ax.axis('off')
    plt.suptitle(f'DPM-Solver: Quality vs Step Count\n"{prompt}"', fontsize=13)
    plt.tight_layout()
    plt.show()

    # YOUR CODE HERE: Create a time-vs-steps plot.
    # Plot step_counts (x-axis) vs step_timings (y-axis).
    # Add axis labels and a title. Add a grid for readability.
    plt.figure(figsize=(8, 4))
    # YOUR CODE HERE
    plt.tight_layout()
    plt.show()

    print(f'\nTime at 5 steps:   {step_timings[0]:.1f}s')
    print(f'Time at 200 steps: {step_timings[-1]:.1f}s')
    print(f'Ratio (200/5): {step_timings[-1] / step_timings[0]:.1f}x')
else:
    print('Fill in the TODOs in the cell above first.')

<details>
<summary>💡 Solution</summary>

The key insight: quality vs steps is **highly nonlinear** for advanced samplers. DPM-Solver achieves good quality by ~20 steps, and adding more steps beyond that yields diminishing returns. This is because the higher-order solver accounts for trajectory curvature, allowing large accurate steps.

**Cell 1 — Generation loop (replace `result = None`):**
```python
result = pipe(
    prompt,
    num_inference_steps=n,
    guidance_scale=guidance_scale,
    generator=generator,
)
```

**Cell 2 — Time-vs-steps plot (replace `# YOUR CODE HERE`):**
```python
plt.plot(step_counts, step_timings, 'o-', color='cyan', linewidth=2, markersize=8)
plt.xlabel('Number of Steps')
plt.ylabel('Generation Time (s)')
plt.title('DPM-Solver: Generation Time vs Step Count')
plt.grid(True, alpha=0.3)
for n, t in zip(step_counts, step_timings):
    plt.annotate(f'{t:.1f}s', (n, t), textcoords='offset points',
                 xytext=(0, 10), ha='center', fontsize=9)
```

**What to observe:**
- 5 steps: likely garbled or very blurry. Even DPM-Solver cannot follow a 1000-timestep trajectory accurately in 5 jumps.
- 10 steps: recognizable but missing fine details. The coarse structure (layout, colors) is established but textures are mushy.
- 20 steps: good quality. The sweet spot for DPM-Solver. This is why 20–30 is the recommended range.
- 50–200 steps: nearly indistinguishable from 20 steps. Diminishing returns.

Generation time scales approximately linearly with step count because each step requires 2 U-Net forward passes (CFG). The fixed overhead (CLIP encoding, VAE decoding, pipeline setup) is small relative to the denoising loop.

</details>

---

## Exercise 4: Inspect DDIM Intermediates [Independent]

The lesson's trajectory perspective says that sampling traces a path from noise to data. DDIM follows a smooth, deterministic trajectory. DDPM follows a jittery, stochastic path. At each step, the latent z_t contains progressively more structure.

Your task:
1. Generate an image with **DDIM at 10 steps**, extracting and storing the **intermediate latent z_t at every step** (10 intermediates total)
2. **VAE-decode each intermediate latent** to visualize the coarse-to-fine progression
3. Generate an image with **DDPM at 10 steps**, extracting the same intermediates
4. Display both sets of intermediates side by side
5. **Reflection:** Why does DDIM at 10 steps produce recognizable intermediates while DDPM at 10 steps looks much worse?

**Key tips:**
- Use `pipe()` with `output_type='latent'` to get the final latent (but you need intermediates at EVERY step, not just the final one)
- To capture intermediates, use the `callback_on_step_end` parameter. The callback receives `(pipe, step_index, timestep, callback_kwargs)` and the current latent is in `callback_kwargs['latents']`
- Or, run the denoising loop manually (like the reference notebook Exercise 3) and store z_t at each step
- Use the `latent_to_pil()` helper to decode latents to images
- Use the same prompt and seed for both DDIM and DDPM

**Expected output:** Two rows of 10 images. The DDIM row should show a smooth progression from noise to a recognizable image. The DDPM row should show a noisier, less coherent progression (because DDPM was designed for 1000 steps, not 10).

In [ ]:
# YOUR CODE HERE
#
# 1. Generate with DDIM at 10 steps, capturing the intermediate latent at every step
# 2. VAE-decode each intermediate to a PIL image
# 3. Generate with DDPM at 10 steps, capturing intermediates the same way
# 4. Display both rows
# 5. Answer the reflection question in a print() statement
#
# Hint for capturing intermediates with a callback:
#
# intermediates = []
# def capture_latent(pipe, step_index, timestep, callback_kwargs):
#     intermediates.append(callback_kwargs['latents'].clone())
#     return callback_kwargs
#
# result = pipe(
#     prompt, num_inference_steps=10, ...,
#     callback_on_step_end=capture_latent,
# )


<details>
<summary>💡 Solution</summary>

The key insight: DDIM at 10 steps shows a **smooth coarse-to-fine progression** because its predict-x₀-then-jump mechanism is designed for arbitrary step sizes. Each step makes a large, accurate leap along the trajectory. DDPM at 10 steps shows **a much less coherent progression** because its reverse formula assumes adjacent timesteps—applying it with gaps of 100 timesteps produces accumulating errors.

```python
prompt = "a mountain landscape with a river"
seed = 42
num_steps = 10

# ---- DDIM intermediates ----
pipe.scheduler = DDIMScheduler.from_config(scheduler_config)

ddim_intermediates = []
def capture_ddim(pipe, step_index, timestep, callback_kwargs):
    ddim_intermediates.append(callback_kwargs['latents'].clone())
    return callback_kwargs

generator = torch.Generator(device=device).manual_seed(seed)
ddim_result = pipe(
    prompt,
    num_inference_steps=num_steps,
    guidance_scale=7.5,
    generator=generator,
    callback_on_step_end=capture_ddim,
)
# The callback fires AFTER each step, so ddim_intermediates[0] is z after step 1,
# and ddim_intermediates[-1] is z_0 (the final latent).

# ---- DDPM intermediates ----
pipe.scheduler = DDPMScheduler.from_config(scheduler_config)

ddpm_intermediates = []
def capture_ddpm(pipe, step_index, timestep, callback_kwargs):
    ddpm_intermediates.append(callback_kwargs['latents'].clone())
    return callback_kwargs

generator = torch.Generator(device=device).manual_seed(seed)
ddpm_result = pipe(
    prompt,
    num_inference_steps=num_steps,
    guidance_scale=7.5,
    generator=generator,
    callback_on_step_end=capture_ddpm,
)

# ---- Decode and display ----
print(f'DDIM intermediates captured: {len(ddim_intermediates)}')
print(f'DDPM intermediates captured: {len(ddpm_intermediates)}')

# Decode each intermediate latent
ddim_decoded = [latent_to_pil(pipe, z) for z in ddim_intermediates]
ddpm_decoded = [latent_to_pil(pipe, z) for z in ddpm_intermediates]

# Display two rows: DDIM on top, DDPM on bottom
fig, axes = plt.subplots(2, num_steps, figsize=(num_steps * 2.5, 5))

for i in range(num_steps):
    axes[0, i].imshow(np.array(ddim_decoded[i]))
    axes[0, i].set_title(f'Step {i+1}', fontsize=8)
    axes[0, i].axis('off')

    axes[1, i].imshow(np.array(ddpm_decoded[i]))
    axes[1, i].set_title(f'Step {i+1}', fontsize=8)
    axes[1, i].axis('off')

axes[0, 0].set_ylabel('DDIM', fontsize=11, rotation=0, labelpad=40)
axes[1, 0].set_ylabel('DDPM', fontsize=11, rotation=0, labelpad=40)

plt.suptitle(f'Intermediate Latents: DDIM vs DDPM at {num_steps} Steps', fontsize=13)
plt.tight_layout()
plt.show()

print()
print('Reflection: Why does DDIM at 10 steps produce recognizable intermediates')
print('while DDPM at 10 steps looks much worse?')
print()
print('DDIM uses the predict-x0-then-jump mechanism: at each step, it predicts')
print('the clean image x0, then uses the closed-form formula to leap to the next')
print('timestep. This formula works with alpha_bar (cumulative signal fraction),')
print('not alpha (single-step), so it handles large jumps accurately.')
print()
print('DDPM uses the reverse step formula calibrated for ADJACENT timesteps.')
print('At 10 steps with 1000 total timesteps, each "step" skips ~100 timesteps.')
print('The DDPM coefficients (alpha_t, beta_t) are calibrated for tiny transitions,')
print('not 100-timestep jumps. Errors accumulate at each step, compounding into')
print('visible artifacts.')
print()
print('Same model, same starting noise, same 10 steps. But DDIM\'s formula was')
print('DESIGNED for arbitrary step sizes. DDPM\'s was not.')
```

**Common mistakes:**
- Forgetting to `.clone()` the latents in the callback. Without cloning, all entries point to the same tensor (the final z_0).
- Using `output_type='latent'` instead of a callback. That only gives you the final latent, not intermediates.
- Not using the same seed for both DDIM and DDPM, making the comparison unfair.

</details>

---

## Key Takeaways

1. **The model predicts noise. The sampler decides what to do with that prediction.** DDPM takes tiny adjacent steps (1000 steps, stochastic). DDIM predicts x₀ and leaps using the closed-form formula (50 steps, deterministic). DPM-Solver reads trajectory curvature with multiple evaluations (20 steps, current standard). All use the exact same trained model.

2. **No retraining. Same weights. Different walkers.** Swapping `pipe.scheduler` is one line of code. The model’s job (predict noise) never changes. The sampler’s job (decide how to step) is the only variable. You verified this by generating comparable images with three different schedulers.

3. **DDIM is deterministic; DDPM is stochastic.** Same seed with DDIM produces pixel-identical images every time (σ=0, no per-step noise injection). DDPM injects fresh noise at every step, producing diverse but non-reproducible results. This is the temperature analogy from Sampling and Generation.

4. **Quality vs steps is highly nonlinear for advanced samplers.** DPM-Solver achieves good quality at ~20 steps. Adding more steps beyond the sweet spot yields diminishing returns. Use the recommended range: DPM-Solver++ at 20–30 steps as your default.

5. **DDIM’s predict-and-leap mechanism handles large step sizes; DDPM’s adjacent-step formula does not.** The intermediate inspection exercise showed this concretely: DDIM at 10 steps produces a smooth coarse-to-fine trajectory, while DDPM at 10 steps produces accumulating artifacts because its formula was calibrated for tiny transitions.